In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import fix_yahoo_finance as yf
import datetime
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [2]:
# end day is today
today = datetime.datetime.today()
today.strftime('%m/%d/%Y')

# start day is 5 business days ago
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
lastBusDay = today - 6 * US_BUSINESS_DAY
lastBusDay.strftime('%m/%d/%Y')

'08/22/2018'

# Calculating daily price movements for all stocks

In [3]:
SP = web.DataReader('^GSPC', 'yahoo', lastBusDay, today)
SP = SP.iloc[::-1]
# generate dollar value of movement between adjacent business days
SP['Mov $'] = round(-SP['Adj Close'].diff().shift(-1), 2)
# generate percentage of movement between adjacent business days
SP['Mov %'] = pd.Series(["{0:.2f}%".format(X) for X in (100*SP['Mov $']/SP['Adj Close'].shift(-1))], index = SP.index)
SP = SP[:5].reset_index()

ticker_list = ['TSLA', 'GOOG', 'AAPL', 'AMZN', 'FB', 'MSFT']
df_final = pd.DataFrame()

for ticker in ticker_list:
    df = web.DataReader(ticker, 'yahoo', lastBusDay, today)
    df['Ticker'] = ticker
    df = df.iloc[::-1]
    # generate dollar value of movement between adjacent business days
    df['Mov $'] = round(-df['Adj Close'].diff().shift(-1), 2)
    # generate percentage of movement between adjacent business days
    df['Mov %'] = pd.Series(["{0:.2f}%".format(X) for X in (100*df['Mov $']/df['Adj Close'].shift(-1))], index = df.index)
    # movement percentage get rid of market fluctuation
    # --> df['Mov_noSP %'] = pd.Series(["{0:.2f}%".format(X) for X in (df['Mov %'] - SP['Mov %'])]) 
    df_final = df_final.append(df[:5])
#len(df_final)
df_final.reset_index()

,Date,High,Low,Open,Close,Volume,Adj Close,Ticker,Mov $,Mov %
0,2018-08-29,311.850006,303.690002,310.269989,305.010010,7426800,305.010010,TSLA,-6.85,-2.20%
1,2018-08-28,318.880005,311.190002,318.410004,311.859985,7649100,311.859985,TSLA,-7.41,-2.32%
2,2018-08-27,322.440002,308.809998,318.000000,319.269989,13079300,319.269989,TSLA,-3.55,-1.10%
3,2018-08-24,323.850006,319.399994,320.700012,322.820007,3602600,322.820007,TSLA,2.72,0.85%
4,2018-08-23,327.320007,318.100006,319.140015,320.100006,5147300,320.100006,TSLA,-1.54,-0.48%
5,2018-08-29,1250.660034,1236.359009,1237.449951,1249.300049,1298900,1249.300049,GOOG,18.15,1.47%
6,2018-08-28,1242.545044,1228.689941,1241.290039,1231.150024,1304000,1231.150024,GOOG,-10.67,-0.86%
7,2018-08-27,1243.089966,1225.715942,1227.599976,1241.819946,1156300,1241.819946,GOOG,21.17,1.73%
8,2018-08-24,1221.650024,1206.359009,1208.819946,1220.650024,946600,1220.650024,GOOG,15.27,1.27%
9,2018-08-23,1221.280029,1204.239990,1207.140015,1205.380005,992600,1205.380005,GOOG,-1.95,-0.16%


# Merge to Articles Data Frame

In [4]:
articles = pd.read_csv("AllArticlesOutput2018-08-28-11-34.csv", encoding = "ISO-8859-1")
articles['DateForm'] = pd.to_datetime(articles['Date']).dt.normalize()
article_w_price_change = (articles.merge(df_final, how='left', left_on=['Ticker','DateForm'], right_on=['Ticker','Date'])).drop(['DateForm','High','Low','Open','Close','Volume','Mov $'], axis=1)
article_w_price_change

,Ticker,Date,Title,Link,Text,Adj Close,Mov %
0,TSLA,08/28/2018 09:32,"Now would be the time for Apple to buy Tesla, ...",http://www.marketwatch.com/story/now-would-be-...,The idea of a merger between Tesla Inc. and Ap...,311.859985,-2.32%
1,TSLA,08/28/2018 09:32,Tesla stock battered anew as analyst questions...,http://www.marketwatch.com/story/tesla-stock-b...,Tesla Inc. shares extended losses to a second ...,311.859985,-2.32%
2,TSLA,08/28/2018 08:32,Its Time for Apple to Buy Tesla and Kick Elon...,https://www.barrons.com/articles/apple-buy-tes...,The idea of a merger between Tesla Inc. and Ap...,311.859985,-2.32%
3,TSLA,08/28/2018 07:32,Why Bondholders Never Bought Elon Musks Talk ...,https://www.barrons.com/articles/why-bondholde...,You can fool some of the people some of the ti...,311.859985,-2.32%
4,TSLA,08/28/2018 05:32,Tesla stock walloped after Musk axes go-privat...,http://www.marketwatch.com/story/tesla-stock-d...,Tesla Inc. shares dropped on Monday after Chie...,311.859985,-2.32%
5,TSLA,08/27/2018 19:32,Teslas Challenges Are Back in Spotlight,https://www.wsj.com/articles/teslas-challenges...,NaN,319.269989,-1.10%
6,TSLA,08/27/18,"Stop Whining, Elon Musk. Tesla Gets a Better D...",https://www.barrons.com/articles/stop-whining-...,Tesla Chief Executive Elon Musk chafes at the ...,319.269989,-1.10%
7,TSLA,08/27/18,Berger Montague Investigating Class Action Cla...,http://www.wsj.com/articles/PR-CO-20180827-904412,NaN,319.269989,-1.10%
8,TSLA,08/27/18,Data Projects the Lithium-Ion Battery Market W...,http://www.wsj.com/articles/PR-CO-20180827-903052,NaN,319.269989,-1.10%
9,TSLA,08/27/18,"Stocks to Watch: Tesla, Netflix, Arconic, PG&E...",https://www.wsj.com/articles/stocks-to-watch-t...,NaN,319.269989,-1.10%
